NOW WE TRAIN AND VALIDATE ON ALL (ITEM, SHOP) EVER SEEN IN DATA
Shold be:
train - all (item ,shop) previosly seen
test - dot product of (item, shop) in current dbn

Now it is as it should be

In [1]:
from sklearn.metrics import root_mean_squared_error

In [2]:
import pandas as pd
import numpy as np


In [3]:
data_train = pd.read_csv('../data_cleaned/data_train.csv')

In [4]:
test = pd.read_csv('../data_cleaned/test.csv')
test['date_block_num'] = 34

In [5]:
data_train = pd.concat([data_train,test ], ignore_index=True).drop('ID', axis=1).fillna(0)

In [6]:
data_train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,06.01.2013,0,25,2554,1709.05,1.0
3,15.01.2013,0,25,2555,1099.00,1.0
4,10.01.2013,0,25,2564,349.00,1.0
...,...,...,...,...,...,...
3142680,0,34,45,18454,0.00,0.0
3142681,0,34,45,16188,0.00,0.0
3142682,0,34,45,15757,0.00,0.0
3142683,0,34,45,19648,0.00,0.0


In [7]:
def prepare_past_ID_s(data_train):
    data_train['shop_item'] = [tuple([shop, item]) for shop, item in zip(data_train['shop_id'], data_train['item_id'])]
    #34 block contains A LOT more shop_item than others
    shop_item_pairs_in_dbn = data_train.groupby('date_block_num')['shop_item'].apply(np.unique)
    data_train = data_train.drop(['shop_item'], axis=1)
    
    shop_item_pairs_WITH_PREV_in_dbn = shop_item_pairs_in_dbn.copy()
    
    print(np.array(shop_item_pairs_WITH_PREV_in_dbn.index))
    arr = np.array(shop_item_pairs_WITH_PREV_in_dbn.index)
    for block in arr[arr>=0]:
        if block == 0:
            continue
        shop_item_pairs_WITH_PREV_in_dbn[block] = np.unique(np.append(shop_item_pairs_WITH_PREV_in_dbn[block -1],
                                                                      #shop_item_pairs_WITH_PREV_in_dbn[block]))
                                                                      shop_item_pairs_in_dbn[block-1]))
        print(len(shop_item_pairs_WITH_PREV_in_dbn[block]))

    return shop_item_pairs_in_dbn, shop_item_pairs_WITH_PREV_in_dbn

In [8]:
shop_item_pairs_in_dbn, shop_item_pairs_WITH_PREV_in_dbn = prepare_past_ID_s(data_train)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34]
63170
97186
126662
146380
162679
178831
192610
205423
216508
228547
240348
255924
263803
271937
281288
290301
299290
307259
314673
322344
330092
338979
349486
362849
368644
374512
380928
386411
391327
396282
401230
406603
411762
418882


In [9]:
pd.concat([shop_item_pairs_WITH_PREV_in_dbn.map(len),shop_item_pairs_in_dbn.map(len)], axis=1)

,shop_item,shop_item
date_block_num,,
0,63170,63170
1,63170,59911
2,97186,63951
3,126662,54590
4,146380,53276
5,162679,56173
6,178831,58004
7,192610,57988
8,205423,51543


In [10]:
from sklearn.metrics import root_mean_squared_error

In [11]:
from collections import defaultdict

In [12]:
def make_X_lag_format(data, dbn):
    """
    transform X to lag format
    columns with dbn in names become lag_0, dbn-1 - lag_1 etc.
    """
    
    lag_cols = defaultdict()
    for col in data.columns:
        splitted = col.split('$')
        
        if len(splitted) == 1:
                continue
        
        
        lag_cols[col] = splitted[0] + '_lag_' + str(dbn - int(splitted[1]))

    #print(lag_cols)
    data = data.rename(columns=dict(lag_cols))
    #print(data.columns)
    return data

In [13]:
def prepare_train(data, valid ):
    """
    returns one batch of merged data with required IDs from valid
    """
    #print(data)
    valid_shop_item = valid
    valid_shop_item = list(zip(*valid_shop_item))
    df = pd.DataFrame({'item_id':valid_shop_item[1],'shop_id':valid_shop_item[0]} )
    data = df.merge(data, on=['shop_id','item_id'], how='left').fillna(0)
    
    return data

In [14]:

def prepare_val(data, valid ):
    """
    returns one batch of merged data with required IDs from valid
    """
    
    df = pd.DataFrame({'item_id':valid[:,1],'shop_id':valid[:,0]} )
    data = df.merge(data, on=['shop_id','item_id'], how='left').fillna(0)
    return data

In [15]:
import re


In [16]:
def prepare_data_train_LSTM(data, valid, dbn):
    """
    
    """
    train = prepare_train (data, valid)
    lag_cols = []
    for col in data.columns:

        splitted = col.split('$')
        if len(splitted)==1:
            lag_cols.append(col)
            continue
        #if 'shop_item_cnt' not in col:
        #    continue
            
        for db in range(0,dbn-1):
            
            if db == int(splitted[1]):
                lag_cols.append(col)

    X = train[lag_cols]

    
    Y = train[f'shop_item_cnt${dbn-1}']
    
    return X, Y
        

In [35]:
def prepare_data_validation_LSTM(data, valid, dbn):
    """
    
    """
    test = prepare_val (data, valid)
    
    lag_cols = []
    
    for col in test.columns:
        
        splitted = col.split('$')
            
        if len(splitted) == 1:
            lag_cols.append(col)
            continue
        #if 'shop_item_cnt' not in col:
        #    continue
        for db in range(1,dbn):
            
            if db == int(splitted[1]):
                #print(db, int(''.join(re.findall(r'\d+', col))))
                lag_cols.append(col)

    X = test[lag_cols]
    Y = test[f'shop_item_cnt${dbn}']
    
    return X, Y

In [36]:
np.random.permutation([[1,2],[3,4]])

array([[1, 2],
       [3, 4]])

In [37]:
def create_batch_train(merged, batch_size, dbn):
    """
    
    """
    #merged = pd.read_csv('data/merged.csv', chunksize=500000)
    #merged = pd.read_csv('data/merged.csv')
    #train = np.random.permutation (shop_item_pairs_WITH_PREV_in_dbn[dbn])
    train = shop_item_pairs_WITH_PREV_in_dbn[dbn]
    chunck_num = (len(train)  // batch_size) + 1
    
    for idx in range(chunck_num):#split shop_item_pairs_WITH_PREV_in_dbn into chuncks
        #for chunck in merged:#split merged into chuncks
        train_ret = prepare_data_train_LSTM(merged,train[idx*batch_size:(idx+1)*batch_size], dbn)
       
        if  train_ret[0].empty:
            yield [None, None]
        
        yield train_ret#, test

In [38]:
def create_batch_val(merged, batch_size, dbn):
    """
    
    """
    #merged = pd.read_csv('data/merged.csv', chunksize=500000) - (DOESNT WORK PROPERLY))))) - use it if merged doesnt fit memory
    #merged = pd.read_csv('data/merged.csv')
    val = shop_item_pairs_in_dbn[dbn]

    shops = np.unique(list(zip(*val))[0])
    items = np.unique(list(zip(*val))[1])

    cartesian_product = np.random.permutation (np.array(np.meshgrid(shops, items)).T.reshape(-1, 2))
    
    chunck_num = (len(cartesian_product)  // batch_size) + 1
    for idx in range(chunck_num):
        #for chunck in merged:
        train_ret = prepare_data_validation_LSTM(merged,cartesian_product[idx*batch_size:(idx+1)*batch_size], dbn)
        #When in batches idx no elements that are in (shop, item) in batch of merged
        if  train_ret[0].empty:
            
            yield [None, None]
        #print(len(train_ret))
        
        yield train_ret#, test

In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim



class CustomLSTM(nn.Module):

    def __init__(self, embedding_dim=1, hidden_dim=64,num_layers=1, tagset_size=1, device=None):
        super(CustomLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim

       
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(input_size=self.embedding_dim,
                            hidden_size = self.hidden_dim,
                            batch_first=True,
                            proj_size=tagset_size,
                            num_layers=num_layers,
                           device=device)

        # The linear layer that maps from hidden state space to tag space
        

    def forward(self, data):
        
        lstm_out, _ = self.lstm(data)
        
        return lstm_out

In [40]:
def preapre_X_LSTM_format(X_train, dbn, device):
    #Batch come here
    #one row - one train example
    cols=[]
    data = defaultdict(list)
    for col in X_train.columns:
        
        if not col[-1].isdigit():
            continue

        if 'shop_item_cnt' not in col:
            continue
        

        cols.append(col)
    #print(list(X_train[cols].columns))

    X_train = X_train[cols].values.reshape(len(X_train), dbn-1, -1)#-1 becuse we dont consider dbn==0
    #print(X_train)
    return torch.tensor(X_train, device=device).to(dtype=torch.float32)
            

In [41]:
def preapre_Y_LSTM_format(Y_train, device):
    return torch.tensor(Y_train, device=device).to(dtype=torch.float32)

In [42]:
def append_some_columns(X_train, dbn):
    X_train['date_block_num'] = dbn
    X_train['month'] = dbn%12
    return X_train

In [43]:
import xgboost as xgb
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression

In [26]:
"""
model=xgb.XGBRegressor(tree_method="hist",
                       #early_stopping_rounds=15,
                       enable_categorical=True,
                       max_depth = 7,
                       n_estimators = 70,
                       learning_rate = 0.05
                       )
"""

#model =RandomForestRegressor(max_depth = 11, n_estimators = 70)
model = LGBMRegressor()

In [27]:

EMBEDDING_DIM = 1

HIDDEN_DIM = 64
TARGET_SIZE=1

device='cuda' if torch.cuda.is_available() else 'cpu'

In [28]:
def train_lstm(model,optimizer,loss_fn,   merged,batch_size, val_month, epochs=20):
    
    first=True
    rmse = 0
    c=0
    
    preds_l=[]
    y_true_l=[]
    for X_train,Y_train  in create_batch_train(merged,batch_size, val_month):
        
        if X_train is None:
            print('None')
            continue
        Y_train = np.clip(Y_train,0,20)
        
        if X_train.empty:
            print('None')
            continue
        X_train = make_X_lag_format(X_train, val_month-1)
        X_train=preapre_X_LSTM_format(X_train, val_month-1, device)
        Y_train = preapre_Y_LSTM_format(Y_train, device)
        
        model.zero_grad()
        
        preds = model(X_train)[:,-1,:]
        
        preds_l.append(torch.squeeze(preds))
        y_true_l.append(torch.squeeze(Y_train))
        
        loss_train = loss_fn(torch.squeeze(preds), 
                             torch.squeeze(Y_train))
        
        
        
        loss_train.backward()
        optimizer.step()
        c+=1

    preds_l = torch.concat(preds_l)
    y_true_l = torch.concat(y_true_l)
    
    with torch.no_grad():
        metric = torch.sqrt(loss_fn(torch.clamp(preds_l,0,20), y_true_l))
        #print(pd.DataFrame(torch.clamp(y_true_l,0,20).numpy(force=True)).describe())
        
    return model, metric

In [29]:
def validate_lstm(model,merged,batch_size, val_month):
    val_error = 0
    c=0
    val_preds=[]
    preds_l=[]
    y_true_l=[]
    #create_batch_train(merged,batch_size, val_month) - return train set, where Y_val
    #is shop_item_cnt_month{val_month}
    loss_fn = nn.MSELoss()

    for X_val, Y_val in create_batch_val(merged,batch_size, val_month):#but then cartesian product used
        
        if X_val is None:
            continue
        if X_val.empty:
            print('None')
            continue
        Y_val = np.clip(Y_val,0,20)        
        X_val = make_X_lag_format(X_val, val_month)
        
        
        X_val=preapre_X_LSTM_format(X_val, val_month, device)
        Y_val = preapre_Y_LSTM_format(Y_val, device)

        #print(X_val.size())
        
        if c==0:
            pass
            #print('train columns',X_val.columns)
            #print(X_val.date_block_num)

        with torch.no_grad():
            y_val_pred = model(X_val)[:,-1,:]
            loss_rmse = torch.sqrt(loss_fn(torch.squeeze(y_val_pred),
                                                 torch.squeeze(Y_val)))
            
            
            preds_l.append(torch.squeeze(y_val_pred))
            y_true_l.append(torch.squeeze(Y_val))
            val_preds.append(y_val_pred)
            c+=1
            
    preds_l = torch.concat(preds_l)
    y_true_l = torch.concat(y_true_l)

    
    with torch.no_grad():
        metric = torch.sqrt(loss_fn(torch.clamp(preds_l,0,20), y_true_l))
        #print(pd.DataFrame(torch.clamp(y_true_l,0,20).numpy(force=True)).describe())
        
            
    return val_preds, metric

In [30]:
def validate_LSTM(merged):
    """
    Function for validating model
    
    """
    
    val_errors = []
    batch_size =15000
    val_preds=[]
    epochs=15
    
    for val_month in range(22, 34):

        model = CustomLSTM(EMBEDDING_DIM, HIDDEN_DIM, TARGET_SIZE,device=device)
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        loss_fn = nn.MSELoss()
        print('date_block_num', val_month)
        print('month', val_month%12)
        
        for epoch in range(epochs):
        
        
            model,train_error = train_lstm(model,
                                           optimizer,
                                           loss_fn, 
                                           merged,
                                           batch_size, 
                                           val_month)
            
            if epoch % 3 == 0:
                val_pred, val_error = validate_lstm(model,
                                                    merged,
                                                    batch_size,
                                                    val_month)
                print('mean train rmse on epoch', epoch,':',train_error )
                print('mean val rmse on epoch', epoch,':',val_error )
                
            
            val_errors.append(val_error)
            val_preds.append(val_pred)
        

    return val_errors, val_preds

In [31]:
chunksize = 50000
l=[]
with pd.read_csv('data/merged.csv', chunksize=chunksize) as reader:
    for chunk in reader:
        l.append(chunk)

merged = pd.concat(l)

In [32]:
list(merged.columns)

['shop_id',
 'item_id',
 'shop_item_cnt$1',
 'shop_item_cnt$2',
 'shop_item_cnt$3',
 'shop_item_cnt$4',
 'shop_item_cnt$5',
 'shop_item_cnt$6',
 'shop_item_cnt$7',
 'shop_item_cnt$8',
 'shop_item_cnt$9',
 'shop_item_cnt$10',
 'shop_item_cnt$11',
 'shop_item_cnt$12',
 'shop_item_cnt$13',
 'shop_item_cnt$14',
 'shop_item_cnt$15',
 'shop_item_cnt$16',
 'shop_item_cnt$17',
 'shop_item_cnt$18',
 'shop_item_cnt$19',
 'shop_item_cnt$20',
 'shop_item_cnt$21',
 'shop_item_cnt$22',
 'shop_item_cnt$23',
 'shop_item_cnt$24',
 'shop_item_cnt$25',
 'shop_item_cnt$26',
 'shop_item_cnt$27',
 'shop_item_cnt$28',
 'shop_item_cnt$29',
 'shop_item_cnt$30',
 'shop_item_cnt$31',
 'shop_item_cnt$32',
 'shop_item_cnt$33',
 'shop_item_cnt$34',
 'item_category_id',
 'shop_item_date_block_num_diff$1',
 'shop_item_date_block_num_diff$2',
 'shop_item_date_block_num_diff$3',
 'shop_item_date_block_num_diff$4',
 'shop_item_date_block_num_diff$5',
 'shop_item_date_block_num_diff$6',
 'shop_item_date_block_num_diff$7'

In [44]:
val_errors, val_preds = validate_LSTM(merged)

date_block_num 22
month 10
mean train rmse on epoch 0 : tensor(1.2602, device='cuda:0')
mean val rmse on epoch 0 : tensor(1.3070, device='cuda:0')


KeyboardInterrupt: 

In [40]:
errors = torch.tensor(val_errors).numpy(force=True).reshape(12,-1)
np.array(errors[:,-1]).mean()

np.float32(0.9546194)

In [52]:
def create_submission(model,merged,batch_size):
    val_month = 34
    test = pd.read_csv('../data_cleaned/test.csv')
    
    data_test = test
    PREDICTION = pd.DataFrame(columns=['shop_id','item_id','item_cnt_month'])
    
    print('date_block_num', val_month)
    print('month', val_month%12)
    for X_val, Y_val in create_batch_val(merged,batch_size, val_month):#but then cartesian product used
        shops= X_val.shop_id
        items = X_val.item_id
        if X_val is None:
            continue
        if X_val.empty:
            print('None')
            continue
        Y_val = np.clip(Y_val,0,20)        
        X_val = make_X_lag_format(X_val, val_month)
        
        
        X_val=preapre_X_LSTM_format(X_val, val_month, device)
        Y_val = preapre_Y_LSTM_format(Y_val, device)

        
        with torch.no_grad():
            y_val_pred = model(X_val)[:,-1,:]
        
            app = pd.DataFrame({'item_id':items,
                                'shop_id': shops,
                                'item_cnt_month':y_val_pred.numpy(force=True).flatten()})
            PREDICTION = pd.concat([PREDICTION, app],ignore_index=True)


    
    data_test = data_test.merge(PREDICTION,on=['shop_id','item_id'])[['ID','item_cnt_month']]
    return data_test
    

In [60]:
def create_submission_pipeline(merged):
    batch_size=20000
    model = CustomLSTM(EMBEDDING_DIM, HIDDEN_DIM, TARGET_SIZE,device=device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()
    val_month=34
    epochs=35
    for epoch in range(epochs):
        print('training epoch',epoch)
        model,columns_order = train_lstm(model,optimizer,loss_fn,   merged,batch_size, val_month, epochs=20)
    
    
    data_test = create_submission(model,merged,batch_size)

    return data_test

In [61]:
submission = create_submission_pipeline(merged)

training epoch 0
training epoch 1
training epoch 2
training epoch 3
training epoch 4
training epoch 5
training epoch 6
training epoch 7
training epoch 8
training epoch 9
training epoch 10
training epoch 11
training epoch 12
training epoch 13
training epoch 14
training epoch 15
training epoch 16
training epoch 17
training epoch 18
training epoch 19
training epoch 20
training epoch 21
training epoch 22
training epoch 23
training epoch 24
training epoch 25
training epoch 26
training epoch 27
training epoch 28
training epoch 29
training epoch 30
training epoch 31
training epoch 32
training epoch 33
training epoch 34
date_block_num 34
month 10


/tmp/ipykernel_11546/2053584585.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  PREDICTION = pd.concat([PREDICTION, app],ignore_index=True)


In [62]:
submission.describe()

,ID,item_cnt_month
count,214200.000000,214200.000000
mean,107099.500000,0.194084
std,61834.358168,0.553474
min,0.000000,0.054468
25%,53549.750000,0.054468
50%,107099.500000,0.054469
75%,160649.250000,0.116074
max,214199.000000,13.267574


In [63]:
submission.to_csv('submission.csv', index=False)

In [ ]:
12 epochs
validation - 0.9546194
test - 1.04
35 epochs
test - 1.03